In [4]:
pip install nvcc4jupyter


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\user\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
%load_ext nvcc4jupyter

Source files will be saved in "C:\Users\user\AppData\Local\Temp\tmph8w0skob".


In [9]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>

#define ROWS 6
#define COLS 7
#define WINNING_COUNT 4

// Kernel to make a move
__global__ void makeMove(int *board, int col, int player) {
    int row = 0;
    while (row < ROWS && board[row * COLS + col] == 0) {
        row++;
    }
    if (row > 0) {
        board[(row - 1) * COLS + col] = player;
    }
}

// Kernel to check for a win
__global__ void checkWin(int *board, bool *winFound) {
    *winFound = false;

    // Check horizontal
    for (int r = 0; r < ROWS; r++) {
        for (int c = 0; c <= COLS - WINNING_COUNT; c++) {
            int count = 1;
            for (int k = 1; k < WINNING_COUNT; k++) {
                if (board[r * COLS + c + k] == board[r * COLS + c]) {
                    count++;
                } else {
                    break;
                }
            }
            if (count == WINNING_COUNT) {
                *winFound = true;
                return;
            }
        }
    }

    // Check vertical
    for (int c = 0; c < COLS; c++) {
        for (int r = 0; r <= ROWS - WINNING_COUNT; r++) {
            int count = 1;
            for (int k = 1; k < WINNING_COUNT; k++) {
                if (board[(r + k) * COLS + c] == board[r * COLS + c]) {
                    count++;
                } else {
                    break;
                }
            }
            if (count == WINNING_COUNT) {
                *winFound = true;
                return;
            }
        }
    }

    // Check diagonals (bottom-left to top-right)
    for (int r = 0; r <= ROWS - WINNING_COUNT; r++) {
        for (int c = 0; c <= COLS - WINNING_COUNT; c++) {
            int count = 1;
            for (int k = 1; k < WINNING_COUNT; k++) {
                if (board[(r + k) * COLS + c + k] == board[r * COLS + c]) {
                    count++;
                } else {
                    break;
                }
            }
            if (count == WINNING_COUNT) {
                *winFound = true;
                return;
            }
        }
    }

    // Check diagonals (top-left to bottom-right)
    for (int r = WINNING_COUNT - 1; r < ROWS; r++) {
        for (int c = 0; c <= COLS - WINNING_COUNT; c++) {
            int count = 1;
            for (int k = 1; k < WINNING_COUNT; k++) {
                if (board[(r - k) * COLS + c + k] == board[r * COLS + c]) {
                    count++;
                } else {
                    break;
                }
            }
            if (count == WINNING_COUNT) {
                *winFound = true;
                return;
            }
        }
    }
}

void printBoard(int *board) {
    for (int r = 0; r < ROWS; r++) {
        for (int c = 0; c < COLS; c++) {
            printf("%d ", board[r * COLS + c]);
        }
        printf("\n");
    }
    printf("\n");
}

int main() {
    int *d_board;
    bool *d_winFound;
    int board[ROWS * COLS] = {0};
    bool winFound = false;

    cudaMalloc(&d_board, sizeof(int) * ROWS * COLS);
    cudaMalloc(&d_winFound, sizeof(bool));

    cudaMemcpy(d_board, board, sizeof(int) * ROWS * COLS, cudaMemcpyHostToDevice);
    cudaMemcpy(d_winFound, &winFound, sizeof(bool), cudaMemcpyHostToDevice);

    // Example moves
    makeMove<<<1, 1>>>(d_board, 3, 1); // Player 1 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 4, 2); // Player 2 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 3, 1); // Player 1 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 4, 2); // Player 2 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 3, 1); // Player 1 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 4, 2); // Player 2 moves
    cudaDeviceSynchronize();
    makeMove<<<1, 1>>>(d_board, 3, 1); // Player 1 moves
    cudaDeviceSynchronize();

    // Check for win
    checkWin<<<1, 1>>>(d_board, d_winFound);
    cudaDeviceSynchronize();

    cudaMemcpy(&winFound, d_winFound, sizeof(bool), cudaMemcpyDeviceToHost);

    if (winFound) {
        printf("Player has won!\n");
    } else {
        printf("No win yet.\n");
    }

    // Print the board
    cudaMemcpy(board, d_board, sizeof(int) * ROWS * COLS, cudaMemcpyDeviceToHost);
    printBoard(board);

    cudaFree(d_board);
    cudaFree(d_winFound);
    return 0;
}


FileNotFoundError: [WinError 2] The system cannot find the file specified